In [293]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objects as go
import scipy.stats as st

In [294]:
def overlap(start1, end1, start2, end2):
    """Does the range (start1, end1) overlap with (start2, end2)?"""
    return (
        start1 <= start2 <= end1 or
        start1 <= end2 <= end1 or
        start2 <= start1 <= end2 or
        start2 <= end1 <= end2
    )

# Data Cleaning for All Datasets

In [295]:
# Read in shark data
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

In [296]:
# Clean eating dataset
eat = eat.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date"}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date


eat.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments
578,2020-09-22,6.0,0.0,1.0,2.0,6.0,0.0,4.0,1.0,0.0,0.0,5.0,1.0,26.0,NaN
579,2020-09-24,0.0,9.0,0.0,7.0,5.0,5.0,7.0,0.0,1.0,2.0,2.0,6.0,44.0,NaN
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,2.0,3.0,3.0,12.0,NaN
581,2020-09-27,5.0,6.0,3.0,6.0,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN
582,2020-09-29,10.0,3.0,1.0,1.0,4.0,2.0,7.0,4.0,4.0,0.0,2.0,0.0,38.0,NaN


In [297]:
# Clean Drops Dataset
drops = drops.iloc[1:,:-1]
drops['Drops'] = pd.to_datetime(drops['Drops']).dt.date

In [298]:
# Clean Targets Dataset
targets = targets.iloc[1:,:-1]
targets.rename(columns = {"Unnamed: 14":'Etc. Comments', "Targets": "Date"}, inplace = True)

#get rid of null values at end of dataset
targets = targets.head(582)

# Change date to just date format, not datetime
targets['Date'] = pd.to_datetime(targets['Date']).dt.date


targets.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:
578,2020-09-22,6.0,1.0,4.0,6.0,5.0,4.0,7.0,2.0,1.0,5.0,4.0,1.0,46.0
579,2020-09-24,8.0,18.0,2.0,9.0,10.0,10.0,11.0,5.0,6.0,3.0,7.0,9.0,98.0
580,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,3.0,5.0,8.0,24.0
581,2020-09-27,10.0,19.0,9.0,10.0,8.0,16.0,3.0,NaN,NaN,NaN,NaN,NaN,75.0
582,2020-09-29,15.0,5.0,6.0,6.0,4.0,4.0,7.0,4.0,1.0,4.0,4.0,1.0,61.0


# Create Subset Groups

In [299]:
# All Sharks Subset
All_Sharks = eat[['Ross', 'Chandler','BT1','BT2','BT3','BT4','BT5','GR1','GR2','GR3','GR4', 'GR5','Total: ']]

# All SS Subset
All_SS = eat[['Ross', 'Chandler']]

# All BT Subset
All_BT = eat[['BT1', 'BT2', 'BT3', 'BT4', 'BT5']]

# All GR Subset
All_GR = eat[['GR1', 'GR2', 'GR3', 'GR4', 'GR5']]

# Male Subset
Male = eat[["BT1","BT5","GR1","Ross","Chandler"]]

# Female Subset
female = eat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]]

# Append all subsets to original datasets
# eat dataset
eat["All_GR"] = eat['GR1'] + eat['GR2'] + eat['GR3'] + eat['GR4'] + eat['GR5']
eat["All_BT"] = eat['BT1'] + eat['BT2'] + eat['BT3'] + eat['BT4'] + eat['BT5']
eat["All_SS"] = eat['Ross'] + eat['Chandler']
eat["male"] = eat[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat["female"] = eat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# drop dataset
drops["All_GR"] = drops['GR1'] + drops['GR2'] + drops['GR3'] + drops['GR4'] + drops['GR5']
drops["All_BT"] = drops['BT1'] + drops['BT2'] + drops['BT3'] + drops['BT4'] + drops['BT5']
drops["All_SS"] = drops['Ross'] + drops['Chandler']
drops["male"] = drops[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops["female"] = drops[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# target dataset
targets["All_GR"] = targets['GR1'] + targets['GR2'] + targets['GR3'] + targets['GR4'] + targets['GR5']
targets["All_BT"] = targets['BT1'] + targets['BT2'] + targets['BT3'] + targets['BT4'] + targets['BT5']
targets["All_SS"] = targets['Ross'] + targets['Chandler']
targets["male"] = targets[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets["female"] = targets[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)


In [300]:
eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,Etc. Comments,All_GR,All_BT,All_SS,male,female
1,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN,6.0,22.0,5.0,12.0,21.0
2,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,0.0,2.0,1.0,1.0,2.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN,6.0,NaN,NaN,2.0,4.0
4,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,8.0,6.0,7.0,7.0
5,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN,7.0,16.0,4.0,9.0,18.0


In [301]:
drops.head()

,Drops,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
1,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,2.0,1.0,0.0,0.0,3.0
2,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,0.0,0.0,10.0,2.0,7.0,1.0,4.0,6.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,1.0
4,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,0.0,1.0,0.0
5,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,2.0,0.0,0.0,3.0


In [302]:
targets.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
1,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,3.0,3.0,0.0,22.0,8.0,11.0,3.0,10.0,12.0
2,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,2.0,2.0,0.0,30.0,4.0,17.0,9.0,16.0,14.0
3,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,0.0,3.0,8.0,8.0,NaN,NaN,1.0,7.0
4,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,15.0,NaN,4.0,11.0,11.0,4.0
5,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,5.0,1.0,3.0,26.0,11.0,11.0,4.0,9.0,17.0


# T test for Shutdown vs. Open Groups

# Number of Pieces Eaten

In [407]:
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
corona_times_eat = (eat['Date'] >= start_date) & (eat['Date'] < end_date)
corona_eat = eat[corona_times_eat]

# Create dataset of the all the time that the Aquarium was open to the public to compare
open_eat_times = (eat['Date'] < start_date) | (eat['Date'] >= end_date)
open_eat = eat[open_eat_times]


open_eat_recent = eat[eat["Date"] < start_date].tail(29)

In [306]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(corona_eat[column], open_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Ross : Test Statistic = -1.61396 P-Value = 0.10726
Chandler : Test Statistic = -0.40546 P-Value = 0.68534
BT1 : Test Statistic = -0.49799 P-Value = 0.61874
BT2 : Test Statistic = 2.87097 P-Value = 0.00429
BT3 : Test Statistic = -1.95872 P-Value = 0.05078
BT4 : Test Statistic = -0.20414 P-Value = 0.83834
BT5 : Test Statistic = -1.04968 P-Value = 0.29445
GR1 : Test Statistic = -1.05797 P-Value = 0.29066
GR2 : Test Statistic = -1.38793 P-Value = 0.16587
GR3 : Test Statistic = -0.11202 P-Value = 0.91086
GR4 : Test Statistic = -0.90367 P-Value = 0.36667
GR5 : Test Statistic = -1.37777 P-Value = 0.16898
Total:  : Test Statistic = -0.81231 P-Value = 0.41695


In [307]:
# Perform T-Test for species and genders
gender_and_species = eat[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species:
    twosample_results = st.ttest_ind(corona_eat[column], open_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

male : Test Statistic = -1.09837 P-Value = 0.2725
female : Test Statistic = -0.1688 P-Value = 0.86602
All_GR : Test Statistic = -1.756 P-Value = 0.07979
All_BT : Test Statistic = -0.24767 P-Value = 0.8045
All_SS : Test Statistic = -1.44172 P-Value = 0.1501


After correcting for the number of T-tests, these are the significant p-values for each subset:

In [308]:
print('Individual Sharks:', round(0.05 / 12, 5))
print('Species:', round(0.05 / 3, 5))
print("Male/Female", round(0.05 / 2, 5))
print("Total:", 0.05)

Individual Sharks: 0.00417
Species: 0.01667
Male/Female 0.025
Total: 0.05


Based on these significant values, there are no significant p-values from any individual shark or subset as a result of our T-tests. We can conclude that there was no significant difference in the number of pieces eaten between the COVID lockdown and the time the Aquarium was open to the public. However, it does look like BT2 was marginally significant and it was just slightly over the updated p-value and should be mentioned.

### Right before corona close down vs. corona close down

In [409]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(corona_eat[column], open_eat_recent[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Ross : Test Statistic = 0.59408 P-Value = 0.55557
Chandler : Test Statistic = -1.06026 P-Value = 0.29495
BT1 : Test Statistic = -0.06089 P-Value = 0.95173
BT2 : Test Statistic = 0.7189 P-Value = 0.47609
BT3 : Test Statistic = -4.20752 P-Value = 0.00013
BT4 : Test Statistic = -0.27066 P-Value = 0.78795
BT5 : Test Statistic = -0.72084 P-Value = 0.47491
GR1 : Test Statistic = -0.78426 P-Value = 0.43729
GR2 : Test Statistic = -0.55232 P-Value = 0.58366
GR3 : Test Statistic = 0.08665 P-Value = 0.93136
GR4 : Test Statistic = 0.28613 P-Value = 0.77618
GR5 : Test Statistic = 0.93299 P-Value = 0.35616
Total:  : Test Statistic = -0.69194 P-Value = 0.49183


In [410]:
# Perform T-Test for species and genders
gender_and_species = eat[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species:
    twosample_results = st.ttest_ind(corona_eat[column], open_eat_recent[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

male : Test Statistic = -0.46032 P-Value = 0.64707
female : Test Statistic = -0.5588 P-Value = 0.57853
All_GR : Test Statistic = -0.03734 P-Value = 0.97039
All_BT : Test Statistic = -1.32614 P-Value = 0.19179
All_SS : Test Statistic = -0.42291 P-Value = 0.67447


BT3 is only significant

# Number of Pieces Dropped

In [412]:
# Create Drops Data for Corona time and open to the public time to perform T-Test
corona_times_drop = (drops['Drops'] >= start_date) & (drops['Drops'] < end_date)
corona_drop = drops[corona_times_drop]

# Create dataset of the all the time that the Aquarium was open to the public to compare
open_drop_times = (drops['Drops'] < start_date) | (drops['Drops'] >= end_date)
open_drop = drops[open_drop_times]


open_drop_recent = drops[drops["Drops"] < start_date].tail(29)

In [310]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

Ross : Test Statistic = 1.58969 P-Value = 0.11263
Chandler : Test Statistic = 0.43535 P-Value = 0.66352
BT1 : Test Statistic = 0.24058 P-Value = 0.80999
BT2 : Test Statistic = 3.89072 P-Value = 0.00012
BT3 : Test Statistic = 1.67011 P-Value = 0.09562
BT4 : Test Statistic = 2.15166 P-Value = 0.03197
BT5 : Test Statistic = 3.07718 P-Value = 0.00222
GR1 : Test Statistic = -0.24089 P-Value = 0.80976
GR2 : Test Statistic = 0.45836 P-Value = 0.64692
GR3 : Test Statistic = 0.60456 P-Value = 0.54579
GR4 : Test Statistic = 0.70405 P-Value = 0.48178
GR5 : Test Statistic = 0.24629 P-Value = 0.80557
Total:  : Test Statistic = 3.82481 P-Value = 0.00015


In [311]:
# Perform T-Test for species and genders
gender_and_species_drop = drops[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species_drop:
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

male : Test Statistic = 2.03828 P-Value = 0.04197
female : Test Statistic = 3.89101 P-Value = 0.00011
All_GR : Test Statistic = 0.74331 P-Value = 0.45769
All_BT : Test Statistic = 4.29426 P-Value = 2e-05
All_SS : Test Statistic = 1.17782 P-Value = 0.23951


Blacktip2 and Blcktip5 have significant p-values for drops, Total, as well as the Blacktip subset and the female subset

### Right before corona drops vs corona drops

In [432]:
# Perform T-Test for each individual Shark
sig = ["Ross","BT5","GR1"]
for column in All_Sharks.drop(columns = sig):
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))
for column in All_Sharks[sig]:
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))


Ross : Test Statistic = 1.58969 P-Value = 0.11263
Chandler : Test Statistic = 0.43535 P-Value = 0.66352
BT1 : Test Statistic = 0.24058 P-Value = 0.80999
BT2 : Test Statistic = 3.89072 P-Value = 0.00012
BT3 : Test Statistic = 1.67011 P-Value = 0.09562
BT4 : Test Statistic = 2.15166 P-Value = 0.03197
BT5 : Test Statistic = 3.07718 P-Value = 0.00222
GR1 : Test Statistic = -0.24089 P-Value = 0.80976
GR2 : Test Statistic = 0.45836 P-Value = 0.64692
GR3 : Test Statistic = 0.60456 P-Value = 0.54579
GR4 : Test Statistic = 0.70405 P-Value = 0.48178
GR5 : Test Statistic = 0.24629 P-Value = 0.80557
Total:  : Test Statistic = 3.82481 P-Value = 0.00015
Ross : Test Statistic = 1.02103 P-Value = 0.31791
BT5 : Test Statistic = 2.3078 P-Value = 0.03124
GR1 : Test Statistic = -0.21216 P-Value = 0.83388


In [415]:
# Perform T-Test for species and genders
sig = ["All_SS"]
gender_and_species_drop = drops[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species_drop.drop(columns = sig):
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))
for column in gender_and_species_drop[sig]:
    twosample_results = st.ttest_ind(corona_drop[column], open_drop[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))
    

male : Test Statistic = 2.03828 P-Value = 0.04197
female : Test Statistic = 3.89101 P-Value = 0.00011
All_GR : Test Statistic = 0.74331 P-Value = 0.45769
All_BT : Test Statistic = 4.29426 P-Value = 2e-05
All_SS : Test Statistic = 0.89045 P-Value = 0.38232


BT2, total, female, all_bt are significant

# Number of Times Targeted

In [416]:
# Create Target Data for Corona time and open to the public time to perform T-Test
corona_times_target = (targets['Date'] >= start_date) & (targets['Date'] < end_date)
corona_target = targets[corona_times_target]

# Create dataset of the all the time that the Aquarium was open to the public to compare
open_target_times = (targets['Date'] < start_date) | (targets['Date'] >= end_date)
open_target = targets[open_target_times]

open_target_recent = targets[targets["Date"] < start_date].tail(29)

In [313]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(corona_target[column], open_target[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

Ross : Test Statistic = -2.2993 P-Value = 0.02196
Chandler : Test Statistic = -1.63313 P-Value = 0.10317
BT1 : Test Statistic = -1.53598 P-Value = 0.12527
BT2 : Test Statistic = 1.64821 P-Value = 0.10003
BT3 : Test Statistic = 0.75754 P-Value = 0.44914
BT4 : Test Statistic = 0.37776 P-Value = 0.70579
BT5 : Test Statistic = -1.69312 P-Value = 0.09115
GR1 : Test Statistic = -1.90131 P-Value = 0.05792
GR2 : Test Statistic = -0.76586 P-Value = 0.44418
GR3 : Test Statistic = -0.93749 P-Value = 0.34903
GR4 : Test Statistic = 0.20691 P-Value = 0.83618
GR5 : Test Statistic = -1.66499 P-Value = 0.09664
Total:  : Test Statistic = -0.84982 P-Value = 0.39578


In [314]:
# Perform T-Test for species and genders
gender_and_species_target = targets[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species_target:
    twosample_results = st.ttest_ind(corona_target[column], open_target[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

male : Test Statistic = -1.65206 P-Value = 0.09906
female : Test Statistic = 0.56898 P-Value = 0.56959
All_GR : Test Statistic = -1.57124 P-Value = 0.11685
All_BT : Test Statistic = -0.27715 P-Value = 0.7818
All_SS : Test Statistic = -2.35684 P-Value = 0.01887


There were no significant p-values in terms of number of pieces dropped between COVID times and regular open times. However, it does seem that Sand Bar Sharks are marginally significant because it was just slightly over the updated p-value and should be mentioned

### Corona targeting corona shut down vs. right before shut down

In [418]:
# Perform T-Test for each individual Shark
sig = ["GR2"]
for column in All_Sharks.drop(columns = sig):
    twosample_results = st.ttest_ind(corona_target[column], open_target_recent[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))
for column in All_Sharks[sig]:
    twosample_results = st.ttest_ind(corona_target[column], open_target_recent[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

Ross : Test Statistic = 0.14935 P-Value = 0.88198
Chandler : Test Statistic = -2.12595 P-Value = 0.03929
BT1 : Test Statistic = -2.10635 P-Value = 0.04105
BT2 : Test Statistic = 0.21887 P-Value = 0.82779
BT3 : Test Statistic = -0.57327 P-Value = 0.56945
BT4 : Test Statistic = -0.55822 P-Value = 0.57959
BT5 : Test Statistic = -1.64764 P-Value = 0.10672
GR1 : Test Statistic = -3.1749 P-Value = 0.00281
GR3 : Test Statistic = -1.39834 P-Value = 0.16935
GR4 : Test Statistic = 0.46115 P-Value = 0.64707
GR5 : Test Statistic = -1.82007 P-Value = 0.07588
Total:  : Test Statistic = -1.32176 P-Value = 0.19162
GR2 : Test Statistic = 0.27252 P-Value = 0.78717


In [419]:
# Perform T-Test for species and genders
gender_and_species_target = targets[['male', 'female', 'All_GR', 'All_BT', 'All_SS']]
for column in gender_and_species_target:
    twosample_results = st.ttest_ind(corona_target[column], open_target_recent[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0],5), "P-Value =", round(twosample_results[1],5))

male : Test Statistic = -1.63827 P-Value = 0.10697
female : Test Statistic = -0.53613 P-Value = 0.59399
All_GR : Test Statistic = -1.60999 P-Value = 0.11489
All_BT : Test Statistic = -1.49725 P-Value = 0.14163
All_SS : Test Statistic = -1.59575 P-Value = 0.11787


GR1 is sig

# T tests for Light Training Vs. No Light Training

Since there were only 41 entries in our data that did not include light training, we will compare this time period with the 40 days immediately after light training began, the final 40 days of our dataset, and to all the days that did not have light training to see if there are significant differences in any of the dates.

# Number of Pieces Eaten

In [315]:
# Create dataset that includes days without light training
start_date = pd.to_datetime('02-15-2018')
#end_date = pd.to_datetime('05-09-2020')
no_light_e = (eat['Date'] < start_date)
no_light_eat = eat[no_light_e]

# Create dataset of first 40 days after light training was introduced
first_40 = (eat["Date"] >= start_date) & (eat["Date"] <= pd.to_datetime('04-28-2018'))
first_40_eat = eat[first_40]

# Create dataset of last 40 days of dataset
last_40 = eat.iloc[-40:]

# Create dataset of all non light training days
light = eat['Date'] >= start_date
light_eat = eat[light]

First we will perform the T-test between the no light training group and the first 40 days after light training was introduced:

In [316]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(no_light_eat[column], first_40_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Ross : Test Statistic = -1.41029 P-Value = 0.16362
Chandler : Test Statistic = -1.97072 P-Value = 0.05337
BT1 : Test Statistic = 0.08784 P-Value = 0.9303
BT2 : Test Statistic = 0.14512 P-Value = 0.8851
BT3 : Test Statistic = 0.62961 P-Value = 0.53134
BT4 : Test Statistic = -0.64733 P-Value = 0.51989
BT5 : Test Statistic = 0.22961 P-Value = 0.81919
GR1 : Test Statistic = 1.15358 P-Value = 0.25317
GR2 : Test Statistic = 0.19689 P-Value = 0.84457
GR3 : Test Statistic = 1.61052 P-Value = 0.11245
GR4 : Test Statistic = 0.02983 P-Value = 0.9763
GR5 : Test Statistic = -1.54532 P-Value = 0.12744
Total:  : Test Statistic = -0.29934 P-Value = 0.76545


In [317]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_eat[column], first_40_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

male : Test Statistic = -1.20314 P-Value = 0.23247
female : Test Statistic = 0.11406 P-Value = 0.90947
All_GR : Test Statistic = 0.57841 P-Value = 0.56512
All_BT : Test Statistic = 0.33383 P-Value = 0.73969
All_SS : Test Statistic = -2.19435 P-Value = 0.03209


There were no significant p-values when conducting the T-tests between the time frame with no light training and the first 40 days of light training. Next we will perform the T-test between the no light training time frame and the last 40 days of the dataset with light training used:

In [318]:
# Perform T-Test for each individual Shark
unequal_var = ["Chandler"]
for column in All_Sharks.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_eat[column], last_40[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))
for column in All_Sharks[unequal_var]:
    twosample_results = st.ttest_ind(no_light_eat[column], last_40[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

Ross : Test Statistic = -1.26515 P-Value = 0.2107924558
BT1 : Test Statistic = -1.12999 P-Value = 0.2630534188
BT2 : Test Statistic = -2.74599 P-Value = 0.0079850899
BT3 : Test Statistic = -1.79207 P-Value = 0.0782491067
BT4 : Test Statistic = -3.42741 P-Value = 0.0011163556
BT5 : Test Statistic = -3.94279 P-Value = 0.0002194297
GR1 : Test Statistic = -3.30711 P-Value = 0.0016085972
GR2 : Test Statistic = -1.72522 P-Value = 0.0897235664
GR3 : Test Statistic = -0.28503 P-Value = 0.7766165661
GR4 : Test Statistic = -2.15341 P-Value = 0.0353844967
GR5 : Test Statistic = -3.87084 P-Value = 0.0002738381
Total:  : Test Statistic = -5.0205 P-Value = 3.1196e-06
Chandler : Test Statistic = -3.12246 P-Value = 0.003305346


In [319]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_eat[column], last_40[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

male : Test Statistic = -4.21881 P-Value = 6e-05
female : Test Statistic = -3.94547 P-Value = 0.00017
All_GR : Test Statistic = -4.64421 P-Value = 2e-05
All_BT : Test Statistic = -4.57361 P-Value = 3e-05
All_SS : Test Statistic = -3.75821 P-Value = 0.00039


Chandler, BT4, BT5, GR1, GR5, males, females, All_GR, All_BT, All_SS, and the total were all significant. This shows evidence that the sharks may have improved over time with the number of the fish they ate because of light training. Now we will compare the time frame of no light training with the whole time frame of light training:

In [320]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(no_light_eat[column], light_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

Ross : Test Statistic = -0.52385 P-Value = 0.6006489928
Chandler : Test Statistic = -1.69831 P-Value = 0.0901602489
BT1 : Test Statistic = -1.33223 P-Value = 0.1834797438
BT2 : Test Statistic = -2.51846 P-Value = 0.0121439573
BT3 : Test Statistic = -0.6316 P-Value = 0.5279772593
BT4 : Test Statistic = -2.93365 P-Value = 0.0035267242
BT5 : Test Statistic = -2.16195 P-Value = 0.0311677576
GR1 : Test Statistic = -1.46375 P-Value = 0.1439840385
GR2 : Test Statistic = -1.34015 P-Value = 0.1808965732
GR3 : Test Statistic = 0.36902 P-Value = 0.7122952273
GR4 : Test Statistic = -1.19187 P-Value = 0.2339604589
GR5 : Test Statistic = -2.40969 P-Value = 0.0163802393
Total:  : Test Statistic = -3.37587 P-Value = 0.0007851421


In [321]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_eat[column], light_eat[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

male : Test Statistic = -2.25547 P-Value = 0.02448
female : Test Statistic = -3.15556 P-Value = 0.00168
All_GR : Test Statistic = -2.18877 P-Value = 0.02914
All_BT : Test Statistic = -3.56903 P-Value = 0.0004
All_SS : Test Statistic = -1.55598 P-Value = 0.12044


The total p-value was significant, BT4 along with the male, female, and All_BT subsets. This is a good indicator that the light training has a significant effect on the number of fish the sharks eat.

# Number of Drops

In [322]:
# Create dataset that includes days without light training

no_light_d = (drops['Drops'] < start_date)
no_light_drops = drops[no_light_d]

# Create dataset of first 40 days after light training was introduced
first_40_d = (drops["Drops"] >= start_date) & (drops["Drops"] <= pd.to_datetime('04-28-2018'))
first_40_drops = drops[first_40_d]

# Create dataset of last 40 days of dataset
last_40_d = drops.iloc[-40:]

# Create dataset of all non light training days
light_d = drops['Drops'] >= start_date
light_drops = drops[light_d]

First we will perform the T-test between the no light training group and the first 40 days after light training was introduced:

In [323]:
# Perform T-Test for each individual Shark
unequal_var = ["GR1","GR2"]
for column in All_Sharks.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], first_40_drops[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in All_Sharks[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], first_40_drops[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Ross : Test Statistic = -0.5234 P-Value = 0.60265
Chandler : Test Statistic = -2.26587 P-Value = 0.02714
BT1 : Test Statistic = 0.60909 P-Value = 0.54476
BT2 : Test Statistic = 0.24832 P-Value = 0.80473
BT3 : Test Statistic = 0.93175 P-Value = 0.3552
BT4 : Test Statistic = -0.63864 P-Value = 0.52549
BT5 : Test Statistic = 0.92287 P-Value = 0.35977
GR3 : Test Statistic = 1.45845 P-Value = 0.14985
GR4 : Test Statistic = 0.94443 P-Value = 0.34867
GR5 : Test Statistic = -0.70028 P-Value = 0.48641
Total:  : Test Statistic = 0.48074 P-Value = 0.63202
GR1 : Test Statistic = 1.89545 P-Value = 0.0653
GR2 : Test Statistic = 1.07753 P-Value = 0.28727


In [324]:
# Perform T-Test for species and genders
unequal_var = ["All_GR"]
for column in gender_and_species.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], first_40_drops[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in gender_and_species[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], first_40_drops[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))


male : Test Statistic = 0.32902 P-Value = 0.743
female : Test Statistic = 1.10857 P-Value = 0.27094
All_BT : Test Statistic = 0.96132 P-Value = 0.34025
All_SS : Test Statistic = -1.9183 P-Value = 0.05992
All_GR : Test Statistic = 1.58527 P-Value = 0.12145


There were no significant P-values between the no light training time frame and the subset of the first 40 days. Now we will compare the last 40 days of the dataset to the days with no light training:

In [325]:
# Perform T-Test for each individual Shark
unequal_var = ["Ross","BT1","GR3","GR4"]
for column in All_Sharks.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], last_40_d[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in All_Sharks[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], last_40_d[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Chandler : Test Statistic = -0.85962 P-Value = 0.39348
BT2 : Test Statistic = -0.22521 P-Value = 0.8226
BT3 : Test Statistic = 1.28264 P-Value = 0.20463
BT4 : Test Statistic = 1.01735 P-Value = 0.31314
BT5 : Test Statistic = 0.16481 P-Value = 0.86965
GR1 : Test Statistic = 1.9847 P-Value = 0.05183
GR2 : Test Statistic = -0.8929 P-Value = 0.37554
GR5 : Test Statistic = -2.03061 P-Value = 0.04681
Total:  : Test Statistic = 1.1526 P-Value = 0.25255
Ross : Test Statistic = 2.49675 P-Value = 0.01825
BT1 : Test Statistic = 1.3336 P-Value = 0.1906
GR3 : Test Statistic = 1.67466 P-Value = 0.10138
GR4 : Test Statistic = 1.61521 P-Value = 0.11389


In [326]:
# Perform T-Test for species and genders
unequal_var = ["male","All_GR"]
for column in gender_and_species.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], last_40_d[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in gender_and_species[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], last_40_d[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

female : Test Statistic = 1.0222 P-Value = 0.30981
All_BT : Test Statistic = 1.46706 P-Value = 0.14767
All_SS : Test Statistic = 0.31344 P-Value = 0.75505
male : Test Statistic = 1.34393 P-Value = 0.18461
All_GR : Test Statistic = 1.05107 P-Value = 0.29952


There were no significant P-values between the subset of no light training and the last 40 days subset. Now we will look at the no light training subset and the light training subset as a whole:

In [327]:
# Perform T-Test for each individual Shark
unequal_var = ["BT1","GR1","GR3"]
for column in All_Sharks.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], light_drops[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in All_Sharks[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], light_drops[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

Ross : Test Statistic = 1.82384 P-Value = 0.06886
Chandler : Test Statistic = 0.36978 P-Value = 0.71173
BT2 : Test Statistic = -0.27582 P-Value = 0.78282
BT3 : Test Statistic = 0.70956 P-Value = 0.47835
BT4 : Test Statistic = -0.18379 P-Value = 0.85427
BT5 : Test Statistic = 0.88521 P-Value = 0.37654
GR2 : Test Statistic = -0.3634 P-Value = 0.71648
GR4 : Test Statistic = 2.04736 P-Value = 0.04122
GR5 : Test Statistic = -0.36774 P-Value = 0.71325
Total:  : Test Statistic = 1.85312 P-Value = 0.06437
BT1 : Test Statistic = 1.15156 P-Value = 0.25837
GR1 : Test Statistic = 2.05487 P-Value = 0.04841
GR3 : Test Statistic = 1.87145 P-Value = 0.07072


In [328]:
# Perform T-Test for species and genders
unequal_var = ["All_GR","male"]
for column in gender_and_species.drop(columns = unequal_var):
    twosample_results = st.ttest_ind(no_light_drops[column], light_drops[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))
for column in gender_and_species[unequal_var]:
    twosample_results = st.ttest_ind(no_light_drops[column], light_drops[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 5))

female : Test Statistic = 1.183 P-Value = 0.23729
All_BT : Test Statistic = 1.44614 P-Value = 0.14886
All_SS : Test Statistic = 1.2498 P-Value = 0.21204
All_GR : Test Statistic = 1.58365 P-Value = 0.12349
male : Test Statistic = 1.54326 P-Value = 0.13042


There are no significant values so this is a good indicator the light training does not significantly effect the number of drops.

# Number of Targets

In [329]:
# Create dataset that includes days without light training

no_light_t = (targets['Date'] < start_date)
no_light_targets = targets[no_light_t]

# Create dataset of first 40 days after light training was introduced
first_40_t = (targets["Date"] >= start_date) & (targets["Date"] <= pd.to_datetime('04-28-2018'))
first_40_targets = drops[first_40_t]

# Create dataset of last 40 days of dataset
last_40_t = targets.iloc[-40:]

# Create dataset of all non light training days
light_t = targets['Date'] >= start_date
light_targets = targets[light_t]

First we will compare the no light training subset to the first 40 days after light training started subset:

In [330]:
# Perform T-Test for each individual Shark

twosample_results = st.ttest_ind(no_light_targets["BT2"], first_40_targets["BT2"],nan_policy='omit')
print("BT2", ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))
for column in All_Sharks.drop(columns = ["BT2"]):
    twosample_results = st.ttest_ind(no_light_targets[column], first_40_targets[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

BT2 : Test Statistic = 6.73226 P-Value = 7.1e-09
Ross : Test Statistic = 7.7305 P-Value = 9.6e-09
Chandler : Test Statistic = 6.9417 P-Value = 6.83e-08
BT1 : Test Statistic = 5.79724 P-Value = 1.568e-06
BT3 : Test Statistic = 8.52927 P-Value = 6e-10
BT4 : Test Statistic = 7.32066 P-Value = 1.56e-08
BT5 : Test Statistic = 6.0582 P-Value = 7.411e-07
GR1 : Test Statistic = 6.41837 P-Value = 3.603e-07
GR2 : Test Statistic = 5.54979 P-Value = 4.2045e-06
GR3 : Test Statistic = 6.91748 P-Value = 8.76e-08
GR4 : Test Statistic = 5.12867 P-Value = 1.41262e-05
GR5 : Test Statistic = 3.027 P-Value = 0.0049112821
Total:  : Test Statistic = 10.35185 P-Value = 0.0


In [331]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_targets[column], first_40_targets[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

male : Test Statistic = 7.89947 P-Value = 9e-10
female : Test Statistic = 10.7562 P-Value = 0.0
All_GR : Test Statistic = 8.87063 P-Value = 5e-10
All_BT : Test Statistic = 9.65337 P-Value = 0.0
All_SS : Test Statistic = 7.97833 P-Value = 4.9e-09


Every single P-value is significant. This would make sense since the light training probably was a shock to the sharks at first and either didn't target as much or targeted more. Now we will compare the non-light training subset to the last 40 days of light training subset:

In [332]:
# Perform T-Test for each individual Shark
for column in All_Sharks:
    twosample_results = st.ttest_ind(no_light_targets[column], last_40_t[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

Ross : Test Statistic = -1.67289 P-Value = 0.0996447128
Chandler : Test Statistic = -1.6297 P-Value = 0.1084922303
BT1 : Test Statistic = 0.50519 P-Value = 0.6153049227
BT2 : Test Statistic = -3.24807 P-Value = 0.0019192862
BT3 : Test Statistic = -1.10217 P-Value = 0.2748626344
BT4 : Test Statistic = -4.15032 P-Value = 0.000108182
BT5 : Test Statistic = -4.10261 P-Value = 0.0001270416
GR1 : Test Statistic = -1.32542 P-Value = 0.1901401276
GR2 : Test Statistic = -2.83406 P-Value = 0.0062826497
GR3 : Test Statistic = 1.69134 P-Value = 0.0960482222
GR4 : Test Statistic = -1.49635 P-Value = 0.1398924537
GR5 : Test Statistic = -3.95334 P-Value = 0.0002088523
Total:  : Test Statistic = -2.70791 P-Value = 0.0082969846


In [333]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_targets[column], last_40_t[column],nan_policy='omit')
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

male : Test Statistic = -1.70236 P-Value = 0.0926199455
female : Test Statistic = -3.41965 P-Value = 0.0009947061
All_GR : Test Statistic = -2.44215 P-Value = 0.0176157187
All_BT : Test Statistic = -3.34471 P-Value = 0.0014361267
All_SS : Test Statistic = -1.86702 P-Value = 0.0668696241


BT2, BT4, BT5, GR2 (marginally), GR5 as well as female, total, All_GR (marginally), and All_BT subsets. Now we will look at the non-light training subset vs. the light training subset as a whole:

In [334]:
# Perform T-Test for each individual Shark
for column in All_Sharks.drop(columns = ["BT2"]):
    twosample_results = st.ttest_ind(no_light_targets[column], light_targets[column],nan_policy='omit',equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))
twosample_results = st.ttest_ind(no_light_targets["BT2"], light_targets["BT2"],nan_policy='omit')
print("BT2", ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

Ross : Test Statistic = -5.31924 P-Value = 3.512e-06
Chandler : Test Statistic = -2.70418 P-Value = 0.0101413491
BT1 : Test Statistic = -3.80233 P-Value = 0.0005213635
BT3 : Test Statistic = -2.92744 P-Value = 0.0056065925
BT4 : Test Statistic = -8.59416 P-Value = 0.0
BT5 : Test Statistic = -5.47487 P-Value = 1.9708e-06
GR1 : Test Statistic = -0.23245 P-Value = 0.8174586026
GR2 : Test Statistic = -3.84724 P-Value = 0.000408231
GR3 : Test Statistic = 0.47432 P-Value = 0.6383120614
GR4 : Test Statistic = -0.8966 P-Value = 0.3758789914
GR5 : Test Statistic = -5.03502 P-Value = 1.14383e-05
Total:  : Test Statistic = -5.83271 P-Value = 3.187e-07
BT2 : Test Statistic = -4.79302 P-Value = 2.2587e-06


In [335]:
# Perform T-Test for species and genders
for column in gender_and_species:
    twosample_results = st.ttest_ind(no_light_targets[column], light_targets[column],nan_policy='omit', equal_var = False)
    print(column, ": Test Statistic =" ,round(twosample_results[0], 5), "P-Value =", round(twosample_results[1], 10))

male : Test Statistic = -4.27758 P-Value = 7.72452e-05
female : Test Statistic = -6.33633 P-Value = 5.34e-08
All_GR : Test Statistic = -2.61123 P-Value = 0.0129375772
All_BT : Test Statistic = -7.67039 P-Value = 2e-09
All_SS : Test Statistic = -4.30354 P-Value = 0.0001118219


All of the P-values are significant, so it seems that targeting was significantly effected by light training.

# Water Temperature Linear Regression to see if temperature had any affect on how much they ate, dropped, and targeted.

In [336]:
from sklearn.linear_model import LinearRegression

In [337]:
model = LinearRegression()
model.fit(np.array(other_factors.Temperature[eat.Chandler.notna().reset_index(drop = True)]).reshape((-1, 1)),eat["Chandler"].dropna())

LinearRegression()

In [338]:
print('intercept:', model.intercept_)
print('slope:', model.coef_)
print("y =",round(model.coef_[0],4),"x -",round(-1*model.intercept_,2))

intercept: -3.4890945546992467
slope: [0.08529267]
y = 0.0853 x - 3.49


In [339]:
model.score(np.array(other_factors.Temperature[eat.Chandler.notna().reset_index(drop = True)]).reshape((-1, 1)),eat["Chandler"].dropna())

0.0019882898707391794

In [340]:
rsq = []
for column in eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    model = LinearRegression()
    model.fit(np.array(other_factors.Temperature[eat[column].notna().reset_index(drop = True)]).reshape((-1, 1)),eat[column].dropna())
    rsq.append(round(model.score(np.array(other_factors.Temperature[eat[column].notna().reset_index(drop = True)]).reshape((-1, 1)),eat[column].dropna()),5))
rsq

[0.00019,
 0.00199,
 0.00105,
 0.00495,
 0.00217,
 0.02497,
 0.00151,
 0.01629,
 0.00262,
 0.00016,
 6e-05,
 0.01364,
 0.00844,
 0.01081,
 0.01838,
 0.00168,
 0.00271,
 0.00995]

In [341]:
rsq = []
for column in drops.iloc[:,1:]:
    model = LinearRegression()
    model.fit(np.array(other_factors.Temperature[drops[column].notna().reset_index(drop = True)]).reshape((-1, 1)),drops[column].dropna())
    rsq.append(round(model.score(np.array(other_factors.Temperature[drops[column].notna().reset_index(drop = True)]).reshape((-1, 1)),drops[column].dropna()),5))
rsq

[0.00282,
 0.00754,
 0.0136,
 0.0157,
 0.01122,
 0.01249,
 0.00187,
 0.00406,
 0.00669,
 0.00203,
 0.01977,
 0.00106,
 0.03424,
 0.01834,
 0.03942,
 0.00912,
 0.01424,
 0.0309]

In [342]:
rsq = []
for column in targets.iloc[:,1:]:
    model = LinearRegression()
    model.fit(np.array(other_factors.Temperature[targets[column].notna().reset_index(drop = True)]).reshape((-1, 1)),targets[column].dropna())
    rsq.append(round(model.score(np.array(other_factors.Temperature[targets[column].notna().reset_index(drop = True)]).reshape((-1, 1)),targets[column].dropna()),5))
rsq

[0.00145,
 0.00146,
 0.00816,
 2e-05,
 3e-05,
 2e-05,
 0.01044,
 0.006,
 0.01467,
 0.0011,
 0.00677,
 0.02721,
 0.00083,
 0.02154,
 1e-05,
 0.00208,
 0.00015,
 0.00203]

In [453]:
px.line(other_factors[["Date","Temperature"]], x = "Date", y = "Temperature", title = "Water Temperature Through Time")

In [344]:
print(other_factors["Temperature"].max())
print(other_factors["Temperature"].min())

78.6
72.0


In [345]:
pd.set_option('display.max_rows', 1000)
other_factors[["Date","Temperature"]]

,Date,Temperature
0,2017-12-05,73.2
1,2017-12-07,73.2
2,2017-12-09,73.6
3,2017-12-10,73.6
4,2017-12-12,73.4
5,2017-12-14,73.0
6,2017-12-16,74.0
7,2017-12-17,73.2
8,2017-12-19,73.2
9,2017-12-21,73.6


# Time of Year Affect on Sharks

In [346]:
#Split the dataframes into different seasons

In [347]:
def season_of_date(date):
    date = pd.to_datetime(date)
    year = str(date.year)
    seasons = {'spring': pd.date_range(start=year+'/03/21', end=year+'/06/20'),
               'summer': pd.date_range(start=year+'/06/21', end=year+'/09/22'),
               'fall': pd.date_range(start=year+'/09/23', end=year+'/12/20')}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['fall']:
        return 'fall'
    else:
        return 'winter'

# Assuming df has a date column of type `datetime`
#df['season'] = df.date.map(season_of_date)

In [348]:
eat_summer = eat[eat.Date.map(season_of_date)=="summer"]
eat_fall = eat[eat.Date.map(season_of_date)=="fall"]
eat_winter = eat[eat.Date.map(season_of_date)=="winter"]
eat_spring = eat[eat.Date.map(season_of_date)=="spring"]

In [349]:
targets_summer = targets[eat.Date.map(season_of_date)=="summer"]
targets_fall = targets[eat.Date.map(season_of_date)=="fall"]
targets_winter = targets[eat.Date.map(season_of_date)=="winter"]
targets_spring = targets[eat.Date.map(season_of_date)=="spring"]

In [350]:
drops_summer = drops[eat.Date.map(season_of_date)=="summer"]
drops_fall = drops[eat.Date.map(season_of_date)=="fall"]
drops_winter = drops[eat.Date.map(season_of_date)=="winter"]
drops_spring = drops[eat.Date.map(season_of_date)=="spring"]

In [351]:
st.f_oneway(eat_summer["Chandler"].dropna(),
               eat_fall["Chandler"].dropna(),
               eat_winter["Chandler"].dropna(),
              eat_spring["Chandler"].dropna())

F_onewayResult(statistic=1.7808321923870167, pvalue=0.1500645155129666)

In [352]:
p_values = {}
for column in drops.iloc[:,1:]:
    p_values.update({column:st.f_oneway(eat_summer[column].dropna(), eat_fall[column].dropna(),eat_winter[column].dropna(),eat_spring[column].dropna())[1]})
p_values = pd.DataFrame(list(p_values.items()),columns = ['Subset','P-Value'])
p_values[p_values["P-Value"]<0.05]

,Subset,P-Value
4,BT3,0.004833
5,BT4,0.004961
6,BT5,0.000216
7,GR1,0.003729
8,GR2,0.038824
10,GR4,0.033529
11,GR5,0.000002
12,Total:,0.000097
13,All_GR,0.000002
14,All_BT,0.000145


In [353]:
p_values = {}
for column in drops.iloc[:,1:]:
    p_values.update({column:st.f_oneway(drops_summer[column].dropna(), drops_fall[column].dropna(),drops_winter[column].dropna(),drops_spring[column].dropna())[1]})
p_values = pd.DataFrame(list(p_values.items()),columns = ['Subset','P-Value'])
p_values[p_values["P-Value"]<0.05]

,Subset,P-Value
2,BT1,0.040907


In [354]:
p_values = {}
for column in drops.iloc[:,1:]:
    p_values.update({column:st.f_oneway(targets_summer[column].dropna(), targets_fall[column].dropna(),targets_winter[column].dropna(),targets_spring[column].dropna())[1]})
p_values = pd.DataFrame(list(p_values.items()),columns = ['Subset','P-Value'])
p_values[p_values["P-Value"]<0.05]

,Subset,P-Value
2,BT1,0.040239
8,GR2,0.000019
11,GR5,0.000846
13,All_GR,0.002038


In [355]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [356]:
test = pd.DataFrame()
sig = ["BT3","BT4","BT5","GR1","GR5","All_GR","All_BT","female","Total: "]
column_names = ["season", "season2"]

for column in sig:
    compare = pd.DataFrame(columns = column_names)
    fall = st.t.interval(alpha=0.95, df=len(eat_fall[column])-1, loc=np.mean(eat_fall[column]), scale=st.sem(eat_fall[column],nan_policy='omit'))
    winter = st.t.interval(alpha=0.95, df=len(eat_winter[column])-1, loc=np.mean(eat_winter[column]), scale=st.sem(eat_winter[column],nan_policy='omit'))
    spring = st.t.interval(alpha=0.95, df=len(eat_spring[column])-1, loc=np.mean(eat_spring[column]), scale=st.sem(eat_spring[column],nan_policy='omit'))
    summer = st.t.interval(alpha=0.95, df=len(eat_summer[column])-1, loc=np.mean(eat_summer[column]), scale=st.sem(eat_summer[column],nan_policy='omit'))   
    for season in [fall, winter, spring, summer]:
        
        for season2 in [fall,winter, spring, summer]:
            if overlap(season[0],season[1],season2[0],season2[1])==False:
                if len(compare[((compare["season"]==namestr(season,globals())[0])&(compare["season2"]==namestr(season2,globals())[0]))|((compare["season"]==namestr(season2,globals())[0])&(compare["season2"]==namestr(season,globals())[0]))])==0:
                    new_row = pd.DataFrame({'season':namestr(season,globals())[0], 'season2':namestr(season2,globals())[0]}, index =[0]) 
                    compare = pd.concat([new_row, compare]).reset_index(drop = True) 
                    print(namestr(season,globals())[0],"and",namestr(season2,globals())[0], "are significantly different for",column)
                    #print(season, "and", season2, "are significantly different for",column)
                
    fall_mean = eat_fall[column].mean()
    winter_mean = eat_winter[column].mean()
    spring_mean = eat_spring[column].mean()
    summer_mean = eat_summer[column].mean()

    fall_error = (fall[1] - fall[0])/2
    winter_error = (winter[1] - winter[0])/2
    spring_error = (spring[1] - spring[0])/2
    summer_error = (summer[1] - summer[0])/2

    pd.Series()
    data = {'Variable':['Fall','Winter','Spring','Summer'], 'Mean':[fall_mean,winter_mean,spring_mean,summer_mean],'error':[fall_error,winter_error,spring_error,summer_error],'Subset':[column,column,column,column]} 
    data = pd.DataFrame(data)
    test = pd.concat([test, data])

test["Mean"] = round(test.Mean,2)
test["error"] = round(test.error,3)

px.bar(test, x = "Subset", y = "Mean", color = "Variable", hover_data = ["error"],error_y = "error", barmode = "group",text = "Mean" ,title = "Subsets that were EATING significantly different between seasons", labels = {"Mean":"Mean of Eating"})
  

fall and winter are significantly different for BT3
fall and summer are significantly different for BT3
winter and summer are significantly different for BT4
fall and spring are significantly different for BT5
winter and summer are significantly different for BT5
spring and summer are significantly different for BT5
winter and summer are significantly different for GR1
spring and summer are significantly different for GR1
winter and summer are significantly different for GR5
spring and summer are significantly different for GR5
winter and summer are significantly different for All_GR
spring and summer are significantly different for All_GR
winter

<ipython-input-356-9732b9c389b8>:31: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



 and summer are significantly different for All_BT
spring and summer are significantly different for All_BT
winter and summer are significantly different for female
spring and summer are significantly different for female
winter and summer are significantly different for Total: 
spring and summer are significantly different for Total: 


In [357]:
test = pd.DataFrame()
sig = ["GR2","GR5","All_GR"]
column_names = ["season", "season2"]

for column in sig:
    compare = pd.DataFrame(columns = column_names)
    fall = st.t.interval(alpha=0.95, df=len(targets_fall[column])-1, loc=np.mean(targets_fall[column]), scale=st.sem(targets_fall[column],nan_policy='omit'))
    winter = st.t.interval(alpha=0.95, df=len(targets_winter[column])-1, loc=np.mean(targets_winter[column]), scale=st.sem(targets_winter[column],nan_policy='omit'))
    spring = st.t.interval(alpha=0.95, df=len(targets_spring[column])-1, loc=np.mean(targets_spring[column]), scale=st.sem(targets_spring[column],nan_policy='omit'))
    summer = st.t.interval(alpha=0.95, df=len(targets_summer[column])-1, loc=np.mean(targets_summer[column]), scale=st.sem(targets_summer[column],nan_policy='omit'))   
    for season in [fall, winter, spring, summer]:
        
        for season2 in [fall,winter, spring, summer]:
            if overlap(season[0],season[1],season2[0],season2[1])==False:
                if len(compare[((compare["season"]==namestr(season,globals())[0])&(compare["season2"]==namestr(season2,globals())[0]))|((compare["season"]==namestr(season2,globals())[0])&(compare["season2"]==namestr(season,globals())[0]))])==0:
                    new_row = pd.DataFrame({'season':namestr(season,globals())[0], 'season2':namestr(season2,globals())[0]}, index =[0]) 
                    compare = pd.concat([new_row, compare]).reset_index(drop = True) 
                    print(namestr(season,globals())[0],"and",namestr(season2,globals())[0], "are significantly different for",column)
                    #print(season, "and", season2, "are significantly different for",column)
                
    fall_mean = targets_fall[column].mean()
    winter_mean = targets_winter[column].mean()
    spring_mean = targets_spring[column].mean()
    summer_mean = targets_summer[column].mean()

    fall_error = (fall[1] - fall[0])/2
    winter_error = (winter[1] - winter[0])/2
    spring_error = (spring[1] - spring[0])/2
    summer_error = (summer[1] - summer[0])/2

    pd.Series()
    data = {'Variable':['Fall','Winter','Spring','Summer'], 'Mean':[fall_mean,winter_mean,spring_mean,summer_mean],'error':[fall_error,winter_error,spring_error,summer_error],'Subset':[column,column,column,column]} 
    data = pd.DataFrame(data)
    test = pd.concat([test, data])

test["Mean"] = round(test.Mean,2)
test["error"] = round(test.error,3)

px.bar(test, x = "Subset", y = "Mean", color = "Variable", error_y = "error", hover_data = ["error"],barmode = "group",text = "Mean" ,title = "Subsets that were TARGETING significantly different between seasons", labels = {"Mean":"Mean of Targeting"})
  

fall and winter are significantly different for GR2
fall and spring are significantly different for GR2
winter and summer are significantly different for GR2
spring and summer are significantly different for GR2
winter and summer are significantly different for GR5
spring and summer are significantly different for GR5
winter and summer are significantly different for All_GR
spring and summer are significantly different for All_GR


<ipython-input-357-a2999275a1c6>:31: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



# Confidence Intervals

## Corona Vs. Non-Corona

In [436]:
print("Corona Eating Mean CI for BT3",st.t.interval(alpha=0.95, df=len(corona_target["GR1"])-1, loc=np.mean(corona_target["GR1"]), scale=st.sem(corona_target["GR1"],nan_policy='omit')))
print("Open Eating Mean CI for BT3",st.t.interval(alpha=0.95, df=len(open_target_recent["GR1"])-1, loc=np.mean(open_target_recent["GR1"]), scale=st.sem(open_target_recent["GR1"],nan_policy='omit')))

Corona Eating Mean CI for BT3 (0.8230882235811623, 2.2678208673279285)
Open Eating Mean CI for BT3 (2.411870575576922, 3.9517657880594412)


In [359]:
print("Corona Dropping Mean CI for BT2",st.t.interval(alpha=0.95, df=len(corona_drop["BT2"])-1, loc=np.mean(corona_drop["BT2"]), scale=st.sem(corona_drop["BT2"],nan_policy='omit')))
print("Open Dropping Mean CI for BT2",st.t.interval(alpha=0.95, df=len(open_drop["BT2"])-1, loc=np.mean(open_drop["BT2"]), scale=st.sem(open_drop["BT2"],nan_policy='omit')))

Corona Dropping Mean CI for BT2 (1.0777505651128656, 2.487466826191482)
Open Dropping Mean CI for BT2 (0.7028128159225746, 0.921283569619594)


In [431]:
test = pd.DataFrame()
sig = ["BT2", "BT5", "female", "All_BT", "Total: "]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_drop[column])-1, loc=np.mean(corona_drop[column]), scale=st.sem(corona_drop[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_drop[column])-1, loc=np.mean(open_drop[column]), scale=st.sem(open_drop[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        #Graph it
        corona_mean = corona_drop[column].mean()
        open_mean = open_drop[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_drop[column])-1, loc=np.mean(corona_drop[column]), scale=st.sem(corona_drop[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_drop[column])-1, loc=np.mean(open_drop[column]), scale=st.sem(open_drop[column],nan_policy='omit'))
        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'Subset':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
test["Mean"] = round(test["Mean"],3)

px.bar(test, x = "Subset", y = "Mean", text = "Mean",color = "Variable", error_y = "error", barmode = "group", title = "Subsets that were significantly different in DROPPING during Corona shutdown vs. non shutdown", labels = {"Mean":"Mean of Dropping"})
  

BT5 Has Confidence Intervals that overlap
Here are the confidence intervals:
Corona (0.6500129331739909, 1.7309394477783902)
Open (0.4887871677833521, 0.6582007840238769)


<ipython-input-431-81778ca324e8>:14: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [434]:
test = pd.DataFrame()
sig = ["BT2","Total: ","female","All_BT"]
#sig = ["Ross","Chandler","BT1","BT2", "BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5","male", "female","All_SS","All_GR", "All_BT", "Total: "]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_drop[column])-1, loc=np.mean(corona_drop[column]), scale=st.sem(corona_drop[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_drop_recent[column])-1, loc=np.mean(open_drop_recent[column]), scale=st.sem(open_drop_recent[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        #Graph it
        corona_mean = corona_drop[column].mean()
        open_mean = open_drop_recent[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_drop[column])-1, loc=np.mean(corona_drop[column]), scale=st.sem(corona_drop[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_drop_recent[column])-1, loc=np.mean(open_drop_recent[column]), scale=st.sem(open_drop_recent[column],nan_policy='omit'))
        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'Subset':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
#test["Mean"] = round(test["Mean"],3)

#px.bar(test, x = "Subset", y = "Mean", text = "Mean",color = "Variable", error_y = "error", barmode = "group", title = "Subsets that were significantly different in DROPPING during Corona shutdown vs. non shutdown", labels = {"Mean":"Mean of Dropping"})
  

BT2 Has Confidence Intervals that overlap
Here are the confidence intervals:
Corona (1.0777505651128656, 2.487466826191482)
Open (0.35843478681413826, 1.2779288495494983)
Total:  Has Confidence Intervals that overlap
Here are the confidence intervals:
Corona (3.79245248868297, 7.448926821661857)
Open (2.774792757907548, 4.742448621402797)
female Has Confidence Intervals that overlap
Here are the confidence intervals:
Corona (2.460949518095134, 4.711464275008314)
Open (1.5446156629639658, 3.214005026691207)
All_BT Has Confidence Intervals that overlap
Here are the confidence intervals:
Corona (4.051819509365043, 7.0910376334921)
Open (3.0516730426084484, 4.675599684664279)


In [443]:
test = pd.DataFrame()
sig = ["All_SS"]
#sig = ["Ross","Chandler","BT1","BT2", "BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5","male", "female","All_SS","All_GR", "All_BT", "Total: "]

for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_target[column])-1, loc=np.mean(corona_target[column]), scale=st.sem(corona_target[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_target[column])-1, loc=np.mean(open_target[column]), scale=st.sem(open_target[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        
        corona_mean = corona_target[column].mean()
        open_mean = open_target[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_target[column])-1, loc=np.mean(corona_target[column]), scale=st.sem(corona_target[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_target[column])-1, loc=np.mean(open_target[column]), scale=st.sem(open_target[column],nan_policy='omit'))

        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
test

px.bar(test, x = "column", y = "Mean", color = "Variable", error_y = "error", barmode = "group", title = "Subsets that were significantly different in Targeting during Corona shutdown vs. non shutdown")
  

<ipython-input-443-10a82961d0ff>:17: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [444]:
test = pd.DataFrame()
sig = ["GR1"]
#Only shows GR1 anyways
#sig = ["Ross","Chandler","BT1","BT2", "BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5","male", "female","All_SS","All_GR", "All_BT", "Total: "]

for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_target[column])-1, loc=np.mean(corona_target[column]), scale=st.sem(corona_target[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_target_recent[column])-1, loc=np.mean(open_target_recent[column]), scale=st.sem(open_target_recent[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        
        corona_mean = corona_target[column].mean()
        open_mean = open_target_recent[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_target[column])-1, loc=np.mean(corona_target[column]), scale=st.sem(corona_target[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_target_recent[column])-1, loc=np.mean(open_target_recent[column]), scale=st.sem(open_target_recent[column],nan_policy='omit'))

        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
test

px.bar(test, x = "column", y = "Mean", color = "Variable", error_y = "error", barmode = "group", title = "Subsets that were significantly different in Targeting during Corona shutdown vs. Right before shutdown")
  

<ipython-input-444-67a86cda850a>:18: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [362]:
test = pd.DataFrame()
sig = ["BT2"]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_eat[column])-1, loc=np.mean(corona_eat[column]), scale=st.sem(corona_eat[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_eat[column])-1, loc=np.mean(open_eat[column]), scale=st.sem(open_eat[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        corona_mean = corona_eat[column].mean()
        open_mean = open_eat[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_eat[column])-1, loc=np.mean(corona_eat[column]), scale=st.sem(corona_eat[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_eat[column])-1, loc=np.mean(open_eat[column]), scale=st.sem(open_eat[column],nan_policy='omit'))
        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'Shark':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
test
test["Mean"] = round(test["Mean"],3)

px.bar(test, x = "Shark", y = "Mean", color = "Variable", text = "Mean",error_y = "error", barmode = "group", title = "Subsets that were significantly different in EATING during Corona shutdown vs. non shutdown", labels = {"Mean":"Mean of Eating"})
  

<ipython-input-362-cf6f3b713549>:13: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [448]:
test = pd.DataFrame()
sig = ["BT3"]
#Only shows BT3 anyways
#sig = ["Ross","Chandler","BT1","BT2", "BT3","BT4","BT5","GR1","GR2","GR3","GR4","GR5","male", "female","All_SS","All_GR", "All_BT", "Total: "]

for column in sig:
    x = st.t.interval(alpha=0.95, df=len(corona_eat[column])-1, loc=np.mean(corona_eat[column]), scale=st.sem(corona_eat[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(open_eat_recent[column])-1, loc=np.mean(open_eat_recent[column]), scale=st.sem(open_eat_recent[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        corona_mean = corona_eat[column].mean()
        open_mean = open_eat_recent[column].mean()
        corona_int = st.t.interval(alpha=0.95, df=len(corona_eat[column])-1, loc=np.mean(corona_eat[column]), scale=st.sem(corona_eat[column],nan_policy='omit'))
        open_int = st.t.interval(alpha=0.95, df=len(open_eat_recent[column])-1, loc=np.mean(open_eat_recent[column]), scale=st.sem(open_eat_recent[column],nan_policy='omit'))
        corona_error = (corona_int[1] - corona_int[0])/2
        open_error = (open_int[1] - open_int[0])/2
        pd.Series()
        data = {'Variable':['Corona','Open'], 'Mean':[corona_mean,open_mean],'error':[corona_error,open_error],'Shark':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Corona",x)
        print("Open",y)
test
test["Mean"] = round(test["Mean"],3)

px.bar(test, x = "Shark", y = "Mean", color = "Variable", text = "Mean",error_y = "error", barmode = "group", title = "Subsets that were significantly different in EATING during Corona Shutdown vs. Right Before Shutdown", labels = {"Mean":"Mean of Eating"})
  

<ipython-input-448-3bea4e27d309>:16: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



## Light Training of first 40 days vs. non-light training

In [363]:
test = pd.DataFrame()
for column in light_targets.iloc[:,1:]:
    x = st.t.interval(alpha=0.95, df=len(first_40_targets[column])-1, loc=np.mean(first_40_targets[column]), scale=st.sem(first_40_targets[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        light_mean = first_40_targets[column].mean()
        no_light_mean = no_light_targets[column].mean()
        light_int = st.t.interval(alpha=0.95, df=len(first_40_targets[column])-1, loc=np.mean(first_40_targets[column]), scale=st.sem(first_40_targets[column],nan_policy='omit'))
        no_light_int = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
        light_error = (light_int[1] - light_int[0])/2
        no_light_error = (no_light_int[1] - no_light_int[0])/2
        pd.Series()
        data = {'Variable':['First 40 Days Of Light Training','Non-Light Training'], 'Mean':[light_mean,no_light_mean],'error':[light_error,no_light_error],'Subset':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Light Training:",x)
        print("Non-Light Training:",y)
test["Mean"] = round(test["Mean"],3)

px.bar(test, x = "Subset", y = "Mean", text = "Mean",color = "Variable", error_y = "error", barmode = "group", category_orders = {"Subset":test.sort_values(by = "Mean", ascending = False)["Subset"]},
       title = "Light Training of first 40 days vs. Non-light training (TARGETING)", labels = {"Mean":"Mean of Targeting"})
  

<ipython-input-363-7a792e265376>:12: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



## Light Training of 40 most recent days vs. non-light training

In [364]:
test = pd.DataFrame()
sig = ["Chandler", "BT2", "BT4","BT5", "GR1", "GR5", "male", "female", "All_BT" ,"All_GR", "All_SS","Total: "]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(last_40[column])-1, loc=np.mean(last_40[column]), scale=st.sem(last_40[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(no_light_eat[column])-1, loc=np.mean(no_light_eat[column]), scale=st.sem(no_light_eat[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        light_mean = last_40[column].mean()
        no_light_mean = no_light_eat[column].mean()
        light_int = st.t.interval(alpha=0.95, df=len(last_40[column])-1, loc=np.mean(last_40[column]), scale=st.sem(last_40[column],nan_policy='omit'))
        no_light_int = st.t.interval(alpha=0.95, df=len(no_light_eat[column])-1, loc=np.mean(no_light_eat[column]), scale=st.sem(no_light_eat[column],nan_policy='omit'))
        light_error = (light_int[1] - light_int[0])/2
        no_light_error = (no_light_int[1] - no_light_int[0])/2
        pd.Series()
        data = {'Variable':['Most Recent 40 Days Of Light Training','Non-Light Training'], 'Mean':[light_mean,no_light_mean],'error':[light_error,no_light_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Light Training:",x)
        print("Non-Light Training:",y)
test["Mean"] = round(test["Mean"],3)

px.bar(test, x = "column", y = "Mean", text = "Mean",color = "Variable", error_y = "error", barmode = "group", title = "Light Training of Most Recent 40 days vs. Non-light training (EATING)",category_orders = {"column":test.sort_values(by = "Mean", ascending = False)["column"]}, labels = {"Mean":"Mean of Eating","column":"Subset"})
#BT2 is marginal 

<ipython-input-364-89b80cadc478>:13: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



BT2 Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (2.3589882272303697, 4.24101177276963)
Non-Light Training: (1.147497719584826, 2.4008893771893676)


In [365]:
test = pd.DataFrame()
sig = ["BT2", "BT4", "BT5", "GR2", "GR5", "female", "All_GR", "All_BT", "Total: "]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(last_40_t[column])-1, loc=np.mean(last_40_t[column]), scale=st.sem(last_40_t[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        light_mean = last_40_t[column].mean()
        no_light_mean = no_light_targets[column].mean()
        light_int = st.t.interval(alpha=0.95, df=len(last_40_t[column])-1, loc=np.mean(last_40_t[column]), scale=st.sem(last_40_t[column],nan_policy='omit'))
        no_light_int = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
        light_error = (light_int[1] - light_int[0])/2
        no_light_error = (no_light_int[1] - no_light_int[0])/2
        pd.Series()
        data = {'Variable':['Most Recent 40 Days Of Light Training','Non-Light Training'], 'Mean':[light_mean,no_light_mean],'error':[light_error,no_light_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Light Training:",x)
        print("Non-Light Training:",y)
test["Mean"] = round(test["Mean"],2)

px.bar(test, x = "column", y = "Mean", color = "Variable",text = "Mean", error_y = "error", barmode = "group", title = "Light Training of Most Recent 40 days vs. Non-light training (TARGETING)",category_orders = {"column":test.sort_values(by = "Mean", ascending = False)["column"]}, labels = {"Mean":"Mean of Targeting","column":"Subset"})
#GR2 and all_gr are marginal 

All_GR Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (15.552234074615262, 20.847765925384735)
Non-Light Training: (10.62276510892794, 16.344976826555932)
Total:  Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (42.436126613591796, 57.51387338640821)
Non-Light Training: (30.45974245957772, 43.15001363798326)


<ipython-input-365-241d14c7f45b>:13: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



## Light Training all-time vs. non-light training

In [366]:
test = pd.DataFrame()
sig = ["BT4", "male", "female","All_BT","Total: "]
for column in sig:
    x = st.t.interval(alpha=0.95, df=len(light_eat[column])-1, loc=np.mean(light_eat[column]), scale=st.sem(light_eat[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(no_light_eat[column])-1, loc=np.mean(no_light_eat[column]), scale=st.sem(no_light_eat[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        light_mean = light_eat[column].mean()
        no_light_mean = no_light_eat[column].mean()
        light_int = st.t.interval(alpha=0.95, df=len(light_eat[column])-1, loc=np.mean(light_eat[column]), scale=st.sem(light_eat[column],nan_policy='omit'))
        no_light_int = st.t.interval(alpha=0.95, df=len(no_light_eat[column])-1, loc=np.mean(no_light_eat[column]), scale=st.sem(no_light_eat[column],nan_policy='omit'))
        light_error = (light_int[1] - light_int[0])/2
        no_light_error = (no_light_int[1] - no_light_int[0])/2
        pd.Series()
        data = {'Variable':['Light Training','Non-Light Training'], 'Mean':[light_mean,no_light_mean],'error':[light_error,no_light_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Light Training:",x)
        print("Non-Light Training:",y)
test["Mean"] = round(test["Mean"],2)

px.bar(test, x = "column", y = "Mean", text = "Mean",color = "Variable", error_y = "error", barmode = "group", title = "Light Training all-time vs. Non-light training (EATING)",category_orders = {"column":test.sort_values(by = "Mean", ascending = False)["column"]}, labels = {"Mean":"Mean of Eating","column":"Subset"})
  

<ipython-input-366-b28e5bb071e8>:13: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [367]:
test = pd.DataFrame()
for column in light_targets.iloc[:,1:]:
    x = st.t.interval(alpha=0.95, df=len(light_targets[column])-1, loc=np.mean(light_targets[column]), scale=st.sem(light_targets[column],nan_policy='omit'))
    y = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
    if overlap(x[0],x[1],y[0],y[1])==False:
        light_mean = light_targets[column].mean()
        no_light_mean = no_light_targets[column].mean()
        light_int = st.t.interval(alpha=0.95, df=len(light_targets[column])-1, loc=np.mean(light_targets[column]), scale=st.sem(light_targets[column],nan_policy='omit'))
        no_light_int = st.t.interval(alpha=0.95, df=len(no_light_targets[column])-1, loc=np.mean(no_light_targets[column]), scale=st.sem(no_light_targets[column],nan_policy='omit'))
        light_error = (light_int[1] - light_int[0])/2
        no_light_error = (no_light_int[1] - no_light_int[0])/2
        pd.Series()
        data = {'Variable':['Light Training','Non-Light Training'], 'Mean':[light_mean,no_light_mean],'error':[light_error,no_light_error],'column':[column,column]} 
        data = pd.DataFrame(data)
        test = pd.concat([test, data])
    else:
        print(column,"Has Confidence Intervals that overlap")
        print("Here are the confidence intervals:")
        print("Light Training:",x)
        print("Non-Light Training:",y)
test["Mean"] = round(test["Mean"],2)

px.bar(test, x = "column", y = "Mean",text = "Mean", color = "Variable", error_y = "error", barmode = "group", title = "Light Training all-time vs. Non-light training (TARGETING)",category_orders = {"column":test.sort_values(by = "Mean", ascending = False)["column"]}, labels = {"Mean":"Mean of Targeting","column":"Subset"})


<ipython-input-367-5a00eb943fd2>:12: DeprecationWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



GR1 Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (2.267731436099342, 2.747046889023811)
Non-Light Training: (1.6947305307234815, 3.1439791466958735)
GR3 Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (4.506700403064565, 5.197733094472381)
Non-Light Training: (3.7832441516943556, 6.6038526224991925)
GR4 Has Confidence Intervals that overlap
Here are the confidence intervals:
Light Training: (2.5600273067538315, 3.085292890290504)
Non-Light Training: (1.5513255931424883, 3.2873840842768667)


# Checking Assumptions

## 1. Check if errors of dist. are normal through qq plots

In [368]:
df = pd.DataFrame()
for column in corona_eat.drop(columns = "Etc. Comments").iloc[:,1:].columns:
    resid = []
    corona_mean = corona_eat[column].mean()
    corona_std = np.std(corona_eat[column])
    open_mean = open_eat[column].mean()
    open_std = np.std(open_eat[column])
    for date in eat.Date:
        if (date >= pd.to_datetime('03-14-2020')) & (date < pd.to_datetime('05-09-2020')):
            resid.append((corona_mean - eat[eat["Date"]==date][column].values[0])/corona_std)
        else:
            resid.append((open_mean - eat[eat["Date"]==date][column].values[0])/open_std)
    df = pd.concat([df, pd.Series(resid,name = column)], axis = 1)
df
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,1.264555,-0.752995,-0.434675,-1.306980,-1.110323,-0.543469,-0.371444,0.994478,0.991042,-0.082193,-0.906527,0.468374,-1.272287,0.485810,-1.468629,0.373629,-0.348729,-1.738560
1,0.913710,1.034939,0.945861,0.315748,1.024914,1.260847,1.287002,0.994478,0.991042,1.094011,1.059721,1.085437,1.829300,1.964620,1.848537,1.366789,1.377323,1.546290
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.466437,0.991042,-0.670295,0.404305,1.085437,1.519141,0.485810,NaN,NaN,1.220409,1.200516
3,-0.489668,0.677352,0.485683,-0.089934,0.170819,0.358689,1.287002,NaN,NaN,NaN,NaN,NaN,0.692051,NaN,0.853387,0.125339,0.435840,0.681856
4,0.562866,0.319765,0.945861,-0.901298,0.170819,-0.543469,-0.786055,0.994478,-0.452054,-0.670295,-0.251111,1.085437,-0.651970,0.239342,-0.473479,0.621919,0.122013,-1.219899
5,-1.893046,1.034939,0.485683,0.721429,1.024914,-0.543469,0.457779,-1.196894,-0.452054,-1.258396,-0.251111,-0.148690,-0.962129,-1.239469,0.853387,-0.619532,-0.819470,-0.701239
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.994478,0.991042,1.094011,1.059721,0.468374,2.036072,1.718152,NaN,NaN,1.534237,1.719177
7,0.913710,0.677352,0.485683,-0.089934,0.597866,0.358689,0.457779,NaN,NaN,NaN,NaN,NaN,1.002210,NaN,0.687529,1.118499,0.749668,0.854742
8,0.562866,-0.037822,-1.355032,0.315748,-0.256228,1.260847,0.043168,-2.657808,0.991042,-0.670295,0.404305,1.085437,-0.651970,-0.253595,0.024096,0.373629,-1.290212,0.336082
9,-0.138823,0.319765,0.945861,1.127111,0.170819,0.809768,0.872391,0.994478,-0.452054,0.505909,1.059721,1.085437,0.795437,1.225215,1.516820,0.125339,0.435840,0.854742


In [369]:
px.histogram(df["female"])

In [370]:
df2 = pd.DataFrame()
for column in light_eat.drop(columns = "Etc. Comments").iloc[:,1:].columns:
    resid = []
    light_mean = light_eat[column].mean()
    light_std = np.std(light_eat[column])
    no_light_mean = no_light_eat[column].mean()
    no_light_std = np.std(no_light_eat[column])
    for date in eat.Date:
        if  (date >= pd.to_datetime('02-15-2018')):
            resid.append((light_mean - eat[eat["Date"]==date][column].values[0])/light_std)
        else:
            resid.append((no_light_mean - eat[eat["Date"]==date][column].values[0])/no_light_std)
    df2 = pd.concat([df2, pd.Series(resid,name = column)], axis = 1)
df2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,1.086337,-1.619330,-1.012836,-2.488139,-1.316713,-1.474221,-1.057497,0.622495,0.857075,-0.021627,-1.368454,0.059976,-2.351244,0.086881,-2.444261,0.109912,-1.020285,-2.313697
1,0.756175,1.138869,1.080358,-0.132954,0.950959,1.029935,1.249769,0.622495,0.857075,1.319247,1.032796,0.989604,1.745116,1.555964,1.435518,1.348921,1.508003,1.125940
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.622495,0.857075,-0.692064,0.232379,0.989604,1.335480,0.086881,NaN,NaN,1.278159,0.763873
3,-0.564469,0.587230,0.382627,-0.721750,0.043890,-0.222143,1.249769,NaN,NaN,NaN,NaN,NaN,0.243117,NaN,0.271585,-0.199840,0.128937,0.220773
4,0.426014,0.035590,1.080358,-1.899343,0.043890,-1.474221,-1.634313,0.622495,-0.803507,-0.692064,-0.568038,0.989604,-1.531972,-0.157966,-1.280327,0.419664,-0.330752,-1.770596
5,-1.885113,1.138869,0.382627,0.455842,0.950959,-1.474221,0.096136,-1.244990,-0.803507,-1.362501,-0.568038,-0.869652,-1.941608,-1.627048,0.271585,-1.129097,-1.709817,-1.227496
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.622495,0.857075,1.319247,1.032796,0.059976,2.018207,1.311117,NaN,NaN,1.737847,1.306974
7,0.756175,0.587230,0.382627,-0.721750,0.497425,-0.222143,0.096136,NaN,NaN,NaN,NaN,NaN,0.652753,NaN,0.077596,1.039169,0.588626,0.401806
8,0.426014,-0.516050,-2.408298,-0.132954,-0.409644,1.029935,-0.480680,-2.489980,0.857075,-0.692064,0.232379,0.989604,-1.531972,-0.647660,-0.698360,0.109912,-2.399350,-0.141294
9,-0.234308,0.035590,1.080358,1.044639,0.043890,0.403896,0.672952,0.622495,-0.803507,0.648810,1.032796,0.989604,0.379663,0.821422,1.047540,-0.199840,0.128937,0.401806


In [371]:
px.histogram(df2["Total: "])

In [372]:
print((light_targets["Chandler"].mean() - light_targets["Chandler"]).mean())
print((no_light_targets["Chandler"].mean() - no_light_targets["Chandler"]).mean())

-7.875473672217859e-16
1.4325458382260084e-16


In [373]:
df3 = pd.DataFrame()
for column in light_targets.iloc[:,1:].columns:
    resid = []
    light_mean = light_targets[column].mean()
    light_std = np.std(light_targets[column])
    no_light_mean = no_light_targets[column].mean()
    no_light_std = np.std(no_light_targets[column])
    for date in eat.Date:
        if  (date < pd.to_datetime('02-15-2018')):
            resid.append((no_light_mean - eat[eat["Date"]==date][column].values[0])/no_light_std)
        else:
            resid.append((light_mean - eat[eat["Date"]==date][column].values[0])/light_std)
    df3 = pd.concat([df3, pd.Series(resid,name = column)], axis = 1)
df3
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,1.513150,0.381797,0.650804,-1.217599,0.156131,-0.068041,-0.091251,1.231994,1.078328,0.835569,-0.246830,0.212000,0.191625,0.965193,0.060043,0.970412,0.504797,-0.310799
1,1.251096,1.457771,1.294684,0.769010,1.769490,1.619385,1.323146,1.231994,1.078328,1.358855,1.028459,0.622749,1.702515,1.739012,1.921376,1.484647,1.332664,1.723761
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.213546,1.078328,0.573927,0.603363,0.622749,1.551426,0.965193,NaN,NaN,1.257404,1.509596
3,0.202881,1.242576,1.080057,0.272358,1.124146,0.775672,1.323146,NaN,NaN,NaN,NaN,NaN,1.148522,NaN,1.362976,0.841853,0.881100,1.188350
4,0.989042,1.027381,1.294684,-0.720947,1.124146,-0.068041,-0.444851,1.231994,-0.035944,0.573927,0.178266,0.622749,0.493803,0.836223,0.618443,1.098970,0.730579,0.010447
5,-0.845335,1.457771,1.080057,1.265662,1.769490,-0.068041,0.615947,-0.295679,-0.035944,0.312284,0.178266,-0.198750,0.342714,0.062405,1.362976,0.456176,0.279015,0.331693
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.231994,1.078328,1.358855,1.028459,0.212000,1.803241,1.610042,NaN,NaN,1.407925,1.830843
7,1.251096,1.242576,1.080057,0.272358,1.446818,0.775672,0.615947,NaN,NaN,NaN,NaN,NaN,1.299611,NaN,1.269910,1.356088,1.031622,1.295432
8,0.989042,0.812187,0.221550,0.769010,0.801475,1.619385,0.262348,-1.314127,1.078328,0.573927,0.603363,0.622749,0.493803,0.578284,0.897643,0.970412,0.053233,0.974186
9,0.464934,1.027381,1.294684,1.762314,1.124146,1.197528,0.969547,1.231994,-0.035944,1.097212,1.028459,0.622749,1.198885,1.352103,1.735243,0.841853,0.881100,1.295432


In [374]:
px.histogram(df3["Total: "])

In [375]:
df4 = pd.DataFrame()
for column in corona_eat.drop(columns = "Etc. Comments").iloc[:,1:].columns:
    resid = []
    fall_mean = eat_fall[column].mean()
    fall_std = np.std(eat_fall[column])
    winter_mean = eat_winter[column].mean()
    winter_std = np.std(eat_winter[column])
    spring_mean = eat_spring[column].mean()
    spring_std = np.std(eat_spring[column])
    summer_mean = eat_summer[column].mean()
    summer_std = np.std(eat_summer[column])

    for date in eat.Date:
        if season_of_date(date)=="fall":
            resid.append((fall_mean - eat[eat["Date"]==date][column].values[0])/fall_std)
        elif season_of_date(date)=='spring':
            resid.append((spring_mean - eat[eat["Date"]==date][column].values[0])/spring_std)
        elif season_of_date(date)=='winter':
            resid.append((winter_mean - eat[eat["Date"]==date][column].values[0])/winter_std)
        else:
            resid.append((summer_mean - eat[eat["Date"]==date][column].values[0])/summer_std)
    df4 = pd.concat([df4, pd.Series(resid,name = column)], axis = 1)
df4
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,1.177911,-0.945253,-0.432298,-1.251537,-1.849485,-0.445368,-0.139539,0.937356,1.060902,0.116916,-0.724186,0.537210,-1.294193,0.625715,-1.494433,0.189694,-0.382346,-1.755338
1,0.804320,1.004904,0.999338,0.284646,0.957769,1.256037,1.372885,0.937356,1.060902,1.344535,1.082695,1.138885,1.843244,2.149194,1.857566,1.229950,1.375303,1.585770
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.548265,1.060902,-0.496893,0.480401,1.138885,1.529500,0.625715,NaN,NaN,1.215517,1.234075
3,-0.690045,0.614873,0.522126,-0.099400,-0.165133,0.405335,1.372885,NaN,NaN,NaN,NaN,NaN,0.692851,NaN,0.851966,-0.070370,0.416586,0.706531
4,0.430729,0.224842,0.999338,-0.867492,-0.165133,-0.445368,-0.517645,0.937356,-0.544788,-0.496893,-0.121893,1.138885,-0.666705,0.371801,-0.488833,0.449758,0.097013,-1.227794
5,-2.184410,1.004904,0.522126,0.668691,0.957769,-0.445368,0.616673,-1.291075,-0.544788,-1.110703,-0.121893,-0.064465,-0.980449,-1.151678,0.851966,-0.850563,-0.861704,-0.700251
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.937356,1.060902,1.344535,1.082695,0.537210,2.052407,1.895281,NaN,NaN,1.535089,1.761618
7,0.804320,0.614873,0.522126,-0.099400,0.396318,0.405335,0.616673,NaN,NaN,NaN,NaN,NaN,1.006594,NaN,0.684366,0.969886,0.736158,0.882379
8,0.430729,-0.165190,-1.386722,0.284646,-0.726583,1.256037,0.238567,-2.776696,1.060902,-0.496893,0.480401,1.138885,-0.666705,-0.136025,0.013967,0.189694,-1.341063,0.354836
9,-0.270501,0.346718,0.880738,0.926086,0.301459,0.598204,0.874089,0.866701,-0.558335,0.473410,1.012603,0.901508,0.787145,0.949419,1.490844,0.061112,0.383426,0.750863


In [376]:
px.histogram(df4["Total: "])

Each of the total histogram of residuals look like they are approximately normal. When we look at the individual sharks they do look like they are left skewed but we are going to say that this assumption passes and we can move on.

## 2. Mean of Residuals needs to be 0

In [377]:
df.mean()

Ross       -1.140502e-16
Chandler    1.918391e-16
BT1         1.138105e-16
BT2        -6.641060e-17
BT3         1.120362e-16
BT4         9.479987e-17
BT5         2.846856e-16
GR1         4.705110e-16
GR2         1.107682e-16
GR3        -5.385979e-17
GR4         1.582766e-16
GR5        -2.342393e-16
Total:      2.025536e-17
All_GR     -6.605446e-18
All_BT      2.495455e-17
All_SS      4.552167e-17
male       -1.442145e-16
female      2.703069e-16
dtype: float64

In [378]:
df2.mean()

Ross        6.459479e-17
Chandler    1.091730e-16
BT1         1.206544e-16
BT2         4.562560e-17
BT3        -4.917426e-17
BT4         3.269835e-17
BT5         1.191707e-16
GR1        -1.971472e-16
GR2         9.349247e-17
GR3        -1.026385e-16
GR4        -6.560986e-17
GR5         1.153412e-16
Total:      7.452444e-17
All_GR      1.498928e-16
All_BT      1.293563e-16
All_SS      1.041941e-16
male       -2.052577e-16
female     -1.564232e-16
dtype: float64

In [379]:
df3.mean()

Ross        0.972752
Chandler    1.006293
BT1         1.277880
BT2         1.001434
BT3         1.084143
BT4         1.119484
BT5         0.757147
GR1         0.480930
GR2         0.688678
GR3         0.847913
GR4         0.447380
GR5         0.609039
Total:      1.202565
All_GR      0.973384
All_BT      1.555910
All_SS      1.185656
male        0.927007
female      1.264122
dtype: float64

In [380]:
df4.mean()

Ross       -5.450186e-17
Chandler    9.957866e-17
BT1        -4.816036e-18
BT2         1.307934e-16
BT3        -4.663951e-17
BT4        -6.210152e-18
BT5         1.568572e-16
GR1         6.300579e-17
GR2         4.166512e-17
GR3         1.625956e-17
GR4         9.336544e-18
GR5         3.810834e-17
Total:      8.866497e-17
All_GR      8.510863e-18
All_BT     -8.912341e-19
All_SS      1.062172e-17
male        3.433679e-17
female      6.562143e-17
dtype: float64

All of the residuals are pretty close to having residuals of 0 so we are good on this assumption

## 3. Do the samples have equal variances?

### Check Corona Vs. Non-Corona

In [381]:
df = pd.DataFrame()
for column in corona_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    corona = corona_eat[column]
    opened = open_eat[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [408]:
df = pd.DataFrame()
for column in corona_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    corona = corona_eat[column]
    opened = open_eat_recent[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [382]:
df = pd.DataFrame()
for column in corona_drop.iloc[:,1:]:
    numbers = []
    corona = corona_drop[column]
    opened = open_drop[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [413]:
df = pd.DataFrame()
for column in corona_drop.iloc[:,1:]:
    numbers = []
    corona = corona_drop[column]
    opened = open_drop_recent[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

<ipython-input-413-b3780ead476f>:7: RuntimeWarning:

divide by zero encountered in double_scalars



,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,False


In [383]:

df = pd.DataFrame()
for column in corona_target.iloc[:,1:]:
    numbers = []
    corona = corona_target[column]
    opened = open_target[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df > 2
    


,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [417]:

df = pd.DataFrame()
for column in corona_target.iloc[:,1:]:
    numbers = []
    corona = corona_target[column]
    opened = open_target_recent[column]
    if np.std(corona)>np.std(opened):
        num = np.std(corona)/np.std(opened)
    else:
        num = np.std(opened)/np.std(corona)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df > 2
    


,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


### Check non-light training vs. all light-training

In [384]:
df = pd.DataFrame()
for column in light_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    light = light_eat[column]
    no_light = no_light_eat[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [385]:
df = pd.DataFrame()
for column in light_drops.iloc[:,1:]:
    numbers = []
    light = light_drops[column]
    no_light = no_light_drops[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,True,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False


Looks like BT1, GR1, GR3, All_GR, and male are all unequal variances. We will need to make sure that we are doing the Welch T-test for these tests.

In [386]:
df = pd.DataFrame()
for column in light_targets.iloc[:,1:]:
    numbers = []
    light = light_targets[column]
    no_light = no_light_targets[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Looks like all of them except BT2 have unequal variances. We will need to make sure we do the Welch T-test.

### Check non light training vs. first 40 days of light training variances

In [387]:
df = pd.DataFrame()
for column in no_light_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    light = first_40_eat[column]
    no_light = no_light_eat[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [388]:
df = pd.DataFrame()
for column in no_light_drops.iloc[:,1:]:
    numbers = []
    light = first_40_drops[column]
    no_light = no_light_drops[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False


Looks like GR1, GR2, and All_GR have unequal variances in these tests

In [389]:
df = pd.DataFrame()
for column in no_light_targets.iloc[:,1:]:
    numbers = []
    light = first_40_targets[column]
    no_light = no_light_targets[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True


All but BT2 have unequal variances here

### Check non light training vs. last 40 days

In [390]:
df = pd.DataFrame()
for column in no_light_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    light = last_40[column]
    no_light = no_light_eat[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Looks like Chandler has unequal variance

In [391]:
df = pd.DataFrame()
for column in no_light_drops.iloc[:,1:]:
    numbers = []
    light = last_40_d[column]
    no_light = no_light_drops[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

<ipython-input-391-cf4a152c5f9d>:9: RuntimeWarning:

divide by zero encountered in double_scalars



,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,True,False,True,False,False,False,False,False,False,True,True,False,False,True,False,False,True,False


Looks like Ross, BT1, GR3, GR4, All_GR, and male all have unequal variances

In [392]:
df = pd.DataFrame()
for column in no_light_targets.iloc[:,1:]:
    numbers = []
    light = last_40_t[column]
    no_light = no_light_targets[column]
    if np.std(light)>np.std(no_light):
        num = np.std(light)/np.std(no_light)
    else:
        num = np.std(no_light)/np.std(light)
    numbers.append(num)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df>2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


There was some definite variances that were not equal as you looked at the two groups. So making note of all of those, I went to the t tests that we made and changed the test to include a parameter that said that the variances were not equal which made some that we found significant, not significant anymore.

## Seasons

In [393]:
df = pd.DataFrame()
for column in corona_eat.drop(columns = "Etc. Comments").iloc[:,1:]:
    numbers = []
    fall = np.std(eat_fall[column])
    winter = np.std(eat_winter[column])
    spring = np.std(eat_spring[column])
    summer = np.std(eat_summer[column])
    max_std = max(fall,winter,spring,summer)
    min_std = min(fall,winter,spring,summer)
    
    numbers.append(max_std/min_std)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [394]:
df = pd.DataFrame()
for column in corona_drop.iloc[:,1:]:
    numbers = []
    fall = np.std(drops_fall[column])
    winter = np.std(drops_winter[column])
    spring = np.std(drops_spring[column])
    summer = np.std(drops_summer[column])
    max_std = max(fall,winter,spring,summer)
    min_std = min(fall,winter,spring,summer)
    
    numbers.append(max_std/min_std)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [395]:
df = pd.DataFrame()
for column in corona_drop.iloc[:,1:]:
    numbers = []
    fall = np.std(targets_fall[column])
    winter = np.std(targets_winter[column])
    spring = np.std(targets_spring[column])
    summer = np.std(targets_summer[column])
    max_std = max(fall,winter,spring,summer)
    min_std = min(fall,winter,spring,summer)
    
    numbers.append(max_std/min_std)
    df = pd.concat([df,pd.Series(numbers,name = column)], axis = 1)
df >2
    

,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total:,All_GR,All_BT,All_SS,male,female
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## 4. Independence Assumption

In [396]:
px.line(df["Ross"].dropna())

In [397]:
px.line(df["Total: "])

In [398]:
px.line(df2["Chandler"].dropna())

In [399]:
px.line(df2["Total: "])

These graphs are looking independent and random so we can say that this assumption also checks out